In [1]:
import bz2
import json
from pathlib import Path

import pandas as pd

my_dir = Path('.')
FULL_DATASET_PATH = "/mnt/e/Uni/Research/lfm2b-data/lyrics_distinct_all_features_non_null_lfm_playcount-2021-12-03.json"

def _stream_lines():
    with open(FULL_DATASET_PATH, 'r') as file:
        for line in file:
            yield json.loads(line)

In [2]:
dic = {'ID': [], 'Genre': [], 'Popularity': [], 'Date': []}
counter = 0

for elem in _stream_lines():
    # print(elem)
    # break
    skip_loop = False

    # get date
    for i in elem['release']:
        if type(elem['release'][i]) is dict:
            skip_loop = True
            counter += 1
            break

        else:
            year, month, n = elem['release'][i].split('-')
            dic['Date'].append(int(year))

    if not skip_loop:
        # get ID
        name = str()
        for i in elem['_id']:
            n = elem['_id'][i]
            n = n.replace(' ', '')
            if len(name) == 0:
                name = n
            else:
                name = name + '_' + n
        dic['ID'].append(name)

        # get popularity
        dic['Popularity'].append(int(elem['popularity']))

        # get genre
        g = elem['genre']
        g = g.replace(' ', '_')
        dic['Genre'].append(g)

        # get features
        for f in elem['features']:
            if type(elem['features'][f]) is dict:
                for e in elem['features'][f]:
                    new_name = f + '_' + e
                    if new_name not in dic:
                        dic[new_name] = []
                        dic[new_name].append(elem['features'][f][e])
                    else:
                        dic[new_name].append(elem['features'][f][e])
            else:
                if f not in dic:
                    dic[f] = []
                    dic[f].append(elem['features'][f])
                else:
                    dic[f].append(elem['features'][f])

        # get affective scores
        for f in elem['affective']:
            if type(elem['affective'][f]) is dict:
                for e in elem['affective'][f]:
                    new_name = f + '_' + e
                    if new_name not in dic:
                        dic[new_name] = []
                        dic[new_name].append(elem['affective'][f][e])
                    else:
                        dic[new_name].append(elem['affective'][f][e])
            else:
                if f not in dic:
                    dic[f] = []
                    dic[f].append(elem['affective'][f])
                else:
                    dic[f].append(elem['affective'][f])
print(counter, 'track have been droped due to invalid date')


8930 track have been droped due to invalid date


In [3]:
# explore missing data
for k, v in dic.items():
    print(k)
    print(len(v))

ID
573829
Genre
573829
Popularity
573829
Date
573829
compression_ratio
573829
entropy
573829
uncommon_words_ratio
573829
readability_flesch_kincaid_grade
573829
readability_flesch_reading_ease
573829
readability_smog
573829
readability_automated_readability_index
573829
readability_coleman_liau_index
573829
readability_dale_chall_readability_score
573829
readability_difficult_words
573829
readability_linsear_write_formula
573829
readability_gunning_fog
573829
readability_fernandez_huerta
573829
readability_szigriszt_pazos
573829
readability_gutierrez_polini
573829
readability_crawford
573829
lexical_token_count
573829
lexical_character_count
573829
lexical_repeated_token_ratio
573829
lexical_unique_tokens_per_line
573829
lexical_average_token_length
573829
lexical_average_tokens_per_line
573829
lexical_line_count
573829
lexical_unique_line_count
573829
lexical_blank_line_count
573829
lexical_blank_line_ratio
573829
lexical_repeated_line_ratio
573829
lexical_exclamation_mark_count
57382

In [4]:
features_df = pd.DataFrame.from_dict(dic)

In [5]:
features_df.to_csv(my_dir / 'features_distinct.csv', index=False)